In [ ]:
# default_exp experiment.mining.ir.unsupervised.d2v

# Neural Unsupervised Approaches for SE Traceability [approach d2v]

> Just Paper. Full Experimentation. This module is dedicated to evaluate word2vec/doc2vec or any neural unsupervised approaches on traceability datasets. Consider to Copy the entire notebook for a new and separeted empirical evaluation. 
> Implementing mutual information analysis
> Author: @danaderp April 2020
> Author: @danielrc Nov 2020

In [ ]:
from ds4se.mining.ir import Doc2VecSeqVect as d2v
from ds4se.mining.ir import VectorizationType
from ds4se.mining.ir import LinkType
from ds4se.mining.ir import SoftwareArtifacts
from ds4se.mining.ir import Preprocessing
from ds4se.mining.ir import DistanceMetric
from ds4se.mining.ir import LoadLinks
from ds4se.mining.ir import SimilarityMetric

In [ ]:
from ds4se.mgmnt.prep.conv import *

In [ ]:
from prg import prg

In [ ]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

# Artifacts Similarity with BasicSequenceVectorization

We test diferent similarities based on [blog](https://www.kdnuggets.com/2017/08/comparing-distance-measurements-python-scipy.html) and [blog2](https://www.kdnuggets.com/2019/01/comparison-text-distance-metrics.html)

# Experients Set-up

In [ ]:
path_data = '../dvc-ds4se/' #dataset path

In [ ]:
#Experiment 3.0.8 <<-- doc2vec
path_to_trained_model = path_data+'models/pv/conv/[doc2vec-Py-Java-PVDBOW-500-20E-1592609630.689167].model'
def smos_params():
        return {
        "vectorizationType": VectorizationType.doc2vec,
        "linkType": LinkType.uc2src,
        "system": 'smos',
        "path_to_trained_model": path_to_trained_model,
        "source_type": SoftwareArtifacts.UC.value,
        "target_type": SoftwareArtifacts.SRC.value,
        "system_path_config": {
            "system_path": path_data + 'se-benchmarking/traceability/testbeds/processed/[smos-all-corpus-1609210822.872445].csv',
            "sep": '~',
            "names": ['ids','conv'],
            "prep": Preprocessing.conv
        },
        "path_mappings": path_data + "se-benchmarking/traceability/testbeds/groundtruth/italian/[smos-ground-uc-to-src].txt",
        "saving_path": path_data + 'metrics/traceability/experiments3.0.x/',
        "names": ['Source','Target','Linked?'],
        }

In [ ]:

parameters = smos_params()
parameters

{'vectorizationType': <VectorizationType.doc2vec: 2>,
 'linkType': <LinkType.uc2src: 5>,
 'system': 'smos',
 'path_to_trained_model': '../dvc-ds4se/models/pv/conv/[doc2vec-Py-Java-PVDBOW-500-20E-1592609630.689167].model',
 'source_type': 'uc',
 'target_type': 'src',
 'system_path_config': {'system_path': '../dvc-ds4se/se-benchmarking/traceability/testbeds/processed/[smos-all-corpus-1609210822.872445].csv',
  'sep': '~',
  'names': ['ids', 'conv'],
  'prep': <Preprocessing.conv: 1>},
 'path_mappings': '../dvc-ds4se/se-benchmarking/traceability/testbeds/groundtruth/italian/[smos-ground-uc-to-src].txt',
 'saving_path': '../dvc-ds4se/metrics/traceability/experiments3.0.x/',
 'names': ['Source', 'Target', 'Linked?']}

# Artifacts Similarity with Doc2Vec

In [ ]:
#[step 1]Creating the Vectorization Class
doc2vec = d2v( params = parameters, logging = logging )

2021-01-14 14:46:27,828 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2021-01-14 14:46:27,860 : INFO : built Dictionary(1499 unique tokens: ['amministrator', 'attori', 'campi', 'caso', 'clicca']...) from 167 documents (total 53746 corpus positions)
2021-01-14 14:46:27,863 : INFO : conventional preprocessing documents, dictionary, and vocab for the test corpus
2021-01-14 14:46:27,866 : INFO : loading Doc2Vec object from ../dvc-ds4se/models/pv/conv/[doc2vec-Py-Java-PVDBOW-500-20E-1592609630.689167].model
2021-01-14 14:46:28,120 : INFO : loading vocabulary recursively from ../dvc-ds4se/models/pv/conv/[doc2vec-Py-Java-PVDBOW-500-20E-1592609630.689167].model.vocabulary.* with mmap=None
2021-01-14 14:46:28,122 : INFO : loading trainables recursively from ../dvc-ds4se/models/pv/conv/[doc2vec-Py-Java-PVDBOW-500-20E-1592609630.689167].model.trainables.* with mmap=None
2021-01-14 14:46:28,125 : INFO : loading syn1neg from ../dvc-ds4se/models/pv/conv/[doc2vec-Py-Java-PVDBOW-500-2

In [ ]:
doc2vec.df_source.head(2)

,ids,conv
0,../dvc-ds4se/se-benchmarking/traceability/data...,nome inserisci nota attori amministrator descr...
1,../dvc-ds4se/se-benchmarking/traceability/data...,nome elimina class attori amministrator descri...


In [ ]:
str(type(doc2vec.new_model))

"<class 'gensim.models.doc2vec.Doc2Vec'>"

In [ ]:
#[step1]Apply Doc2Vec Inference
doc2vec.InferDoc2Vec( steps = 200 )

2021-01-14 14:46:56,538 : INFO : Infer Doc2Vec on Source and Target Complete


In [ ]:
doc2vec.df_inferred_src.head(2)

,ids,conv,inf-doc2vec
0,../dvc-ds4se/se-benchmarking/traceability/data...,nome inserisci nota attori amministrator descr...,"[1.2688141, -0.6409156, -0.7696343, -0.9540485..."
1,../dvc-ds4se/se-benchmarking/traceability/data...,nome elimina class attori amministrator descri...,"[0.24126159, -0.12495557, -0.1083778, -0.31200..."


In [ ]:
#[step 2]NonGroundTruth Computation
metric_l = [DistanceMetric.EUC,DistanceMetric.COS,DistanceMetric.MAN, SimilarityMetric.Pearson]
doc2vec.ComputeDistanceArtifacts( sampling=False, samples = 50, metric_list = metric_l )
doc2vec.df_nonground_link.head()

2021-01-14 14:46:56,576 : INFO : Computed distances or similarities ('../dvc-ds4se/se-benchmarking/traceability/datasets/formatted/semeru_format/SMOS_semeru_format/use_cases/SMOS37.txt', '../dvc-ds4se/se-benchmarking/traceability/datasets/formatted/semeru_format/SMOS_semeru_format/source_code/ManagerClassroom.java')[[99.7626724243164, 0.009924310024142199], [0.8408116847276688, 0.15918831527233124], [1797.5583, 0.0005560008660365052], [0.16037117550838192]]
2021-01-14 14:46:56,580 : INFO : Computed distances or similarities ('../dvc-ds4se/se-benchmarking/traceability/datasets/formatted/semeru_format/SMOS_semeru_format/use_cases/SMOS37.txt', '../dvc-ds4se/se-benchmarking/traceability/datasets/formatted/semeru_format/SMOS_semeru_format/source_code/ServletUpdateUser.java')[[62.85104751586914, 0.015661450186098613], [0.8608479797840118, 0.13915202021598816], [1108.9575, 0.0009009353803218644], [0.13941506166721426]]
2021-01-14 14:46:56,585 : INFO : Computed distances or similarities ('../d

2021-01-14 14:46:56,661 : INFO : Computed distances or similarities ('../dvc-ds4se/se-benchmarking/traceability/datasets/formatted/semeru_format/SMOS_semeru_format/use_cases/SMOS37.txt', '../dvc-ds4se/se-benchmarking/traceability/datasets/formatted/semeru_format/SMOS_semeru_format/source_code/ConnectionWrapper.java')[[77.37580871582031, 0.012759038999212878], [0.935889221727848, 0.06411077827215195], [1411.063, 0.0007081837058962863], [0.06528328385204421]]
2021-01-14 14:46:56,665 : INFO : Computed distances or similarities ('../dvc-ds4se/se-benchmarking/traceability/datasets/formatted/semeru_format/SMOS_semeru_format/use_cases/SMOS37.txt', '../dvc-ds4se/se-benchmarking/traceability/datasets/formatted/semeru_format/SMOS_semeru_format/source_code/DeleteManagerException.java')[[27.42513084411621, 0.03518013709361667], [0.75666743516922, 0.24333256483078003], [488.16794, 0.002044287701302416], [0.2456785223997199]]
2021-01-14 14:46:56,670 : INFO : Computed distances or similarities ('../d

2021-01-14 14:46:56,747 : INFO : Computed distances or similarities ('../dvc-ds4se/se-benchmarking/traceability/datasets/formatted/semeru_format/SMOS_semeru_format/use_cases/SMOS37.txt', '../dvc-ds4se/se-benchmarking/traceability/datasets/formatted/semeru_format/SMOS_semeru_format/source_code/UserListItem.java')[[30.27971649169922, 0.031969599221443476], [0.837410181760788, 0.16258981823921204], [542.7353, 0.0018391302117435604], [0.16306108440926767]]
2021-01-14 14:46:56,751 : INFO : Computed distances or similarities ('../dvc-ds4se/se-benchmarking/traceability/datasets/formatted/semeru_format/SMOS_semeru_format/use_cases/SMOS37.txt', '../dvc-ds4se/se-benchmarking/traceability/datasets/formatted/semeru_format/SMOS_semeru_format/source_code/ServletDeleteReport.java')[[59.959922790527344, 0.016404220251988108], [0.8989685699343681, 0.10103143006563187], [1083.3574, 0.0009222051510200994], [0.10166943651730058]]
2021-01-14 14:46:56,756 : INFO : Computed distances or similarities ('../dvc

2021-01-14 14:46:56,832 : INFO : Computed distances or similarities ('../dvc-ds4se/se-benchmarking/traceability/datasets/formatted/semeru_format/SMOS_semeru_format/use_cases/SMOS37.txt', '../dvc-ds4se/se-benchmarking/traceability/datasets/formatted/semeru_format/SMOS_semeru_format/source_code/ServletShowUserDetails.java')[[60.43553161621094, 0.01627722547022171], [0.9134991392493248, 0.0865008607506752], [1089.4783, 0.0009170288176753723], [0.08724038403426378]]
2021-01-14 14:46:56,836 : INFO : Computed distances or similarities ('../dvc-ds4se/se-benchmarking/traceability/datasets/formatted/semeru_format/SMOS_semeru_format/use_cases/SMOS37.txt', '../dvc-ds4se/se-benchmarking/traceability/datasets/formatted/semeru_format/SMOS_semeru_format/source_code/ServletUpdateRegister.java')[[73.49674987792969, 0.013423404398696576], [0.8275125026702881, 0.17248749732971191], [1319.8469, 0.0007570900018465159], [0.17321127967756933]]
2021-01-14 14:46:56,841 : INFO : Computed distances or similariti

2021-01-14 14:46:56,920 : INFO : Computed distances or similarities ('../dvc-ds4se/se-benchmarking/traceability/datasets/formatted/semeru_format/SMOS_semeru_format/use_cases/SMOS37.txt', '../dvc-ds4se/se-benchmarking/traceability/datasets/formatted/semeru_format/SMOS_semeru_format/source_code/ServletShowReports.java')[[63.53881072998047, 0.015494552637231446], [0.9141964614391327, 0.08580353856086731], [1124.2168, 0.0008887176255964563], [0.08619431319971553]]
2021-01-14 14:46:56,925 : INFO : Computed distances or similarities ('../dvc-ds4se/se-benchmarking/traceability/datasets/formatted/semeru_format/SMOS_semeru_format/use_cases/SMOS37.txt', '../dvc-ds4se/se-benchmarking/traceability/datasets/formatted/semeru_format/SMOS_semeru_format/source_code/ServletShowNoteDetails.java')[[66.0517807006836, 0.014913847023153032], [0.8798703327775002, 0.12012966722249985], [1184.217, 0.0008437273219958848], [0.12021156330343316]]
2021-01-14 14:46:56,929 : INFO : Computed distances or similarities 

...

2021-01-14 14:47:17,879 : INFO : Computed distances or similarities ('../dvc-ds4se/se-benchmarking/traceability/datasets/formatted/semeru_format/SMOS_semeru_format/use_cases/SMOS27.txt', '../dvc-ds4se/se-benchmarking/traceability/datasets/formatted/semeru_format/SMOS_semeru_format/source_code/ServletShowUserTeachingForm.java')[[65.57957458496094, 0.015019621351348812], [0.9387810863554478, 0.06121891364455223], [1153.2318, 0.0008663770916867459], [0.05998726148930523]]
2021-01-14 14:47:17,882 : INFO : Computed distances or similarities ('../dvc-ds4se/se-benchmarking/traceability/datasets/formatted/semeru_format/SMOS_semeru_format/use_cases/SMOS27.txt', '../dvc-ds4se/se-benchmarking/traceability/datasets/formatted/semeru_format/SMOS_semeru_format/source_code/ServletShowTeachingDetails.java')[[61.3906364440918, 0.01602804614593248], [0.923667699098587, 0.07633230090141296], [1089.8368, 0.0009167274218663885], [0.07578762904693996]]
2021-01-14 14:47:17,885 : INFO : Computed distances or s

2021-01-14 14:47:17,933 : INFO : Computed distances or similarities ('../dvc-ds4se/se-benchmarking/traceability/datasets/formatted/semeru_format/SMOS_semeru_format/use_cases/SMOS27.txt', '../dvc-ds4se/se-benchmarking/traceability/datasets/formatted/semeru_format/SMOS_semeru_format/source_code/AdaptorException.java')[[31.06780242919922, 0.031183926688080555], [0.880887970328331, 0.119112029671669], [552.76575, 0.0018058177221875525], [0.1161393831179183]]
2021-01-14 14:47:17,936 : INFO : Computed distances or similarities ('../dvc-ds4se/se-benchmarking/traceability/datasets/formatted/semeru_format/SMOS_semeru_format/use_cases/SMOS27.txt', '../dvc-ds4se/se-benchmarking/traceability/datasets/formatted/semeru_format/SMOS_semeru_format/source_code/ServletLoadTeachingList.java')[[64.17436981201172, 0.015343454840981043], [0.9232403114438057, 0.0767596885561943], [1143.2542, 0.0008739317219506002], [0.07514086796165056]]
2021-01-14 14:47:17,939 : INFO : Computed distances or similarities ('..

2021-01-14 14:47:17,985 : INFO : Computed distances or similarities ('../dvc-ds4se/se-benchmarking/traceability/datasets/formatted/semeru_format/SMOS_semeru_format/use_cases/SMOS27.txt', '../dvc-ds4se/se-benchmarking/traceability/datasets/formatted/semeru_format/SMOS_semeru_format/source_code/ServletInsertClassroom.java')[[66.84176635742188, 0.014740182246015476], [0.9315284863114357, 0.0684715136885643], [1179.6823, 0.0008469679282236037], [0.0676240774626883]]
2021-01-14 14:47:17,988 : INFO : Computed distances or similarities ('../dvc-ds4se/se-benchmarking/traceability/datasets/formatted/semeru_format/SMOS_semeru_format/use_cases/SMOS27.txt', '../dvc-ds4se/se-benchmarking/traceability/datasets/formatted/semeru_format/SMOS_semeru_format/source_code/ServletAssignParentStudent.java')[[67.58978271484375, 0.014579430935907989], [0.8539059609174728, 0.14609403908252716], [1212.9565, 0.0008237527165136275], [0.1446589526352473]]
2021-01-14 14:47:17,991 : INFO : Computed distances or simila

2021-01-14 14:47:18,034 : INFO : Computed distances or similarities ('../dvc-ds4se/se-benchmarking/traceability/datasets/formatted/semeru_format/SMOS_semeru_format/use_cases/SMOS27.txt', '../dvc-ds4se/se-benchmarking/traceability/datasets/formatted/semeru_format/SMOS_semeru_format/source_code/Classroom.java')[[40.73844909667969, 0.023958724428971425], [0.6864306330680847, 0.3135693669319153], [736.4958, 0.0013559399463599295], [0.31303908384486484]]
2021-01-14 14:47:18,037 : INFO : Computed distances or similarities ('../dvc-ds4se/se-benchmarking/traceability/datasets/formatted/semeru_format/SMOS_semeru_format/use_cases/SMOS27.txt', '../dvc-ds4se/se-benchmarking/traceability/datasets/formatted/semeru_format/SMOS_semeru_format/source_code/UserTest.java')[[39.59934997558594, 0.024630936224381453], [0.9823573920875788, 0.017642607912421227], [703.43164, 0.0014195841616551465], [0.01679458385928439]]
2021-01-14 14:47:18,040 : INFO : Computed distances or similarities ('../dvc-ds4se/se-benc

,Source,Target,DistanceMetric.EUC,SimilarityMetric.EUC_sim,DistanceMetric.COS,SimilarityMetric.COS_sim,DistanceMetric.MAN,SimilarityMetric.MAN_sim,SimilarityMetric.Pearson
0,../dvc-ds4se/se-benchmarking/traceability/data...,../dvc-ds4se/se-benchmarking/traceability/data...,99.762672,0.009924,0.840812,0.159188,1797.558350,0.000556,0.160371
1,../dvc-ds4se/se-benchmarking/traceability/data...,../dvc-ds4se/se-benchmarking/traceability/data...,62.851048,0.015661,0.860848,0.139152,1108.957520,0.000901,0.139415
2,../dvc-ds4se/se-benchmarking/traceability/data...,../dvc-ds4se/se-benchmarking/traceability/data...,67.854279,0.014523,0.889312,0.110688,1223.475952,0.000817,0.111629
3,../dvc-ds4se/se-benchmarking/traceability/data...,../dvc-ds4se/se-benchmarking/traceability/data...,27.217810,0.035439,0.942001,0.057999,492.634827,0.002026,0.058512
4,../dvc-ds4se/se-benchmarking/traceability/data...,../dvc-ds4se/se-benchmarking/traceability/data...,38.018795,0.025629,0.820864,0.179136,674.813660,0.001480,0.180110


In [ ]:
#[step 3]Saving Non-GroundTruth Links
doc2vec.SaveLinks()

2021-01-14 14:47:18,199 : INFO : Saving in...../dvc-ds4se/metrics/traceability/experiments3.0.x/[smos-VectorizationType.doc2vec-LinkType.uc2src-False-1610635638.08545].csv


In [ ]:
#[step 4]GroundTruthMatching Testing
path_to_ground_truth = parameters['path_mappings']

doc2vec.MatchWithGroundTruth(path_to_ground_truth, semeru_format= True)
doc2vec.df_ground_link

2021-01-14 14:47:18,214 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:18,221 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:18,227 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:18,234 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:18,239 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:18,245 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:18,252 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:18,257 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:18,263 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:18,269 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:18,275 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:18,281 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:18,287 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:18,293 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:18,299 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:18,305 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:18,311 

2021-01-14 14:47:19,030 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:19,036 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:19,042 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:19,048 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:19,054 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:19,060 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:19,066 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:19,072 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:19,078 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:19,084 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:19,090 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:19,096 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:19,101 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:19,107 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:19,113 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:19,119 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:19,125 

2021-01-14 14:47:19,840 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:19,846 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:19,851 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:19,857 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:19,863 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:19,869 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:19,875 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:19,881 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:19,887 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:19,893 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:19,899 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:19,905 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:19,912 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:19,918 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:19,924 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:19,930 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:19,936 

2021-01-14 14:47:20,682 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:20,689 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:20,697 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:20,704 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:20,712 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:20,720 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:20,728 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:20,735 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:20,743 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:20,750 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:20,758 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:20,767 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:20,775 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:20,782 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:20,790 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:20,797 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:20,805 

2021-01-14 14:47:21,675 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:21,682 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:21,689 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:21,696 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:21,704 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:21,711 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:21,718 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:21,725 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:21,732 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:21,740 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:21,747 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:21,754 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:21,761 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:21,768 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:21,776 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:21,783 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:21,790 

2021-01-14 14:47:22,693 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:22,700 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:22,708 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:22,715 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:22,723 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:22,730 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:22,737 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:22,744 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:22,752 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:22,759 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:22,766 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:22,773 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:22,780 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:22,787 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:22,795 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:22,802 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:22,809 

2021-01-14 14:47:23,672 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:23,679 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:23,686 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:23,693 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:23,700 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:23,708 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:23,715 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:23,722 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:23,729 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:23,736 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:23,744 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:23,751 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:23,759 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:23,768 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:23,775 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:23,783 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:23,790 

2021-01-14 14:47:24,685 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:24,694 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:24,702 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:24,709 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:24,716 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:24,724 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:24,731 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:24,739 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:24,746 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:24,753 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:24,761 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:24,768 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:24,776 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:24,783 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:24,791 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:24,798 : INFO : findDistInDF: semeru_format
2021-01-14 14:47:24,806 

,Source,Target,DistanceMetric.EUC,SimilarityMetric.EUC_sim,DistanceMetric.COS,SimilarityMetric.COS_sim,DistanceMetric.MAN,SimilarityMetric.MAN_sim,SimilarityMetric.Pearson,Linked?
0,../dvc-ds4se/se-benchmarking/traceability/data...,../dvc-ds4se/se-benchmarking/traceability/data...,99.762672,0.009924,0.840812,0.159188,1797.558350,0.000556,0.160371,0.0
1,../dvc-ds4se/se-benchmarking/traceability/data...,../dvc-ds4se/se-benchmarking/traceability/data...,62.851048,0.015661,0.860848,0.139152,1108.957520,0.000901,0.139415,0.0
2,../dvc-ds4se/se-benchmarking/traceability/data...,../dvc-ds4se/se-benchmarking/traceability/data...,67.854279,0.014523,0.889312,0.110688,1223.475952,0.000817,0.111629,1.0
3,../dvc-ds4se/se-benchmarking/traceability/data...,../dvc-ds4se/se-benchmarking/traceability/data...,27.217810,0.035439,0.942001,0.057999,492.634827,0.002026,0.058512,0.0
4,../dvc-ds4se/se-benchmarking/traceability/data...,../dvc-ds4se/se-benchmarking/traceability/data...,38.018795,0.025629,0.820864,0.179136,674.813660,0.001480,0.180110,0.0
...,...,...,...,...,...,...,...,...,...,...
6695,../dvc-ds4se/se-benchmarking/traceability/data...,../dvc-ds4se/se-benchmarking/traceability/data...,63.234657,0.015568,0.936642,0.063358,1126.772461,0.000887,0.062838,0.0
6696,../dvc-ds4se/se-benchmarking/traceability/data...,../dvc-ds4se/se-benchmarking/traceability/data...,65.932289,0.014940,0.888382,0.111618,1178.674561,0.000848,0.110528,0.0
6697,../dvc-ds4se/se-benchmarking/traceability/data...,../dvc-ds4se/se-benchmarking/traceability/data...,44.590954,0.021934,0.872361,0.127639,808.769287,0.001235,0.125207,0.0
6698,../dvc-ds4se/se-benchmarking/traceability/data...,../dvc-ds4se/se-benchmarking/traceability/data...,69.409218,0.014203,0.934687,0.065313,1231.119629,0.000812,0.065008,0.0


In [ ]:
doc2vec.df_ground_link[doc2vec.df_ground_link ['Linked?']==1]

,Source,Target,DistanceMetric.EUC,SimilarityMetric.EUC_sim,DistanceMetric.COS,SimilarityMetric.COS_sim,DistanceMetric.MAN,SimilarityMetric.MAN_sim,SimilarityMetric.Pearson,Linked?
2,../dvc-ds4se/se-benchmarking/traceability/data...,../dvc-ds4se/se-benchmarking/traceability/data...,67.854279,0.014523,0.889312,0.110688,1223.475952,0.000817,0.111629,1.0
13,../dvc-ds4se/se-benchmarking/traceability/data...,../dvc-ds4se/se-benchmarking/traceability/data...,64.535713,0.015259,0.873459,0.126541,1157.908447,0.000863,0.126462,1.0
14,../dvc-ds4se/se-benchmarking/traceability/data...,../dvc-ds4se/se-benchmarking/traceability/data...,64.428482,0.015284,0.943787,0.056213,1147.924683,0.000870,0.057659,1.0
32,../dvc-ds4se/se-benchmarking/traceability/data...,../dvc-ds4se/se-benchmarking/traceability/data...,66.466805,0.014822,0.917298,0.082702,1176.520508,0.000849,0.084489,1.0
34,../dvc-ds4se/se-benchmarking/traceability/data...,../dvc-ds4se/se-benchmarking/traceability/data...,63.781364,0.015437,0.928793,0.071207,1132.268555,0.000882,0.072490,1.0
...,...,...,...,...,...,...,...,...,...,...
6673,../dvc-ds4se/se-benchmarking/traceability/data...,../dvc-ds4se/se-benchmarking/traceability/data...,65.770889,0.014977,0.896260,0.103740,1176.443848,0.000849,0.103696,1.0
6675,../dvc-ds4se/se-benchmarking/traceability/data...,../dvc-ds4se/se-benchmarking/traceability/data...,64.512054,0.015264,0.948828,0.051172,1130.742920,0.000884,0.049166,1.0
6678,../dvc-ds4se/se-benchmarking/traceability/data...,../dvc-ds4se/se-benchmarking/traceability/data...,39.285610,0.024823,0.741942,0.258058,696.422852,0.001434,0.256786,1.0
6680,../dvc-ds4se/se-benchmarking/traceability/data...,../dvc-ds4se/se-benchmarking/traceability/data...,45.724438,0.021402,0.658281,0.341719,814.241150,0.001227,0.343608,1.0


In [ ]:
#[step 4.1]GroundTruthMatching Testing For CISCO Mappings <--- Only SACP
doc2vec.MatchWithGroundTruth(from_mappings=True)
doc2vec.df_ground_link

In [ ]:
#[step 5]Saving GroundTruth Links
doc2vec.SaveLinks(grtruth = True)

2021-01-14 14:48:36,746 : INFO : Saving in...../dvc-ds4se/metrics/traceability/experiments3.0.x/[smos-VectorizationType.doc2vec-LinkType.uc2src-True-1610635716.624465].csv


In [ ]:
#Loading Non-GroundTruth Links (change the timestamp with the assigned in the previous step)
df_glinks = LoadLinks(timestamp=1610635716.624465, params=parameters,grtruth = True, logging = logging)
df_glinks.head()

2021-01-14 14:48:58,750 : INFO : Loading computed links from... ../dvc-ds4se/metrics/traceability/experiments3.0.x/[smos-VectorizationType.doc2vec-LinkType.uc2src-True-1610635716.624465].csv


,Source,Target,DistanceMetric.EUC,SimilarityMetric.EUC_sim,DistanceMetric.COS,SimilarityMetric.COS_sim,DistanceMetric.MAN,SimilarityMetric.MAN_sim,SimilarityMetric.Pearson,Linked?
0,../dvc-ds4se/se-benchmarking/traceability/data...,../dvc-ds4se/se-benchmarking/traceability/data...,99.762672,0.009924,0.840812,0.159188,1797.558350,0.000556,0.160371,0.0
1,../dvc-ds4se/se-benchmarking/traceability/data...,../dvc-ds4se/se-benchmarking/traceability/data...,62.851048,0.015661,0.860848,0.139152,1108.957520,0.000901,0.139415,0.0
2,../dvc-ds4se/se-benchmarking/traceability/data...,../dvc-ds4se/se-benchmarking/traceability/data...,67.854279,0.014523,0.889312,0.110688,1223.475952,0.000817,0.111629,1.0
3,../dvc-ds4se/se-benchmarking/traceability/data...,../dvc-ds4se/se-benchmarking/traceability/data...,27.217810,0.035439,0.942001,0.057999,492.634827,0.002026,0.058512,0.0
4,../dvc-ds4se/se-benchmarking/traceability/data...,../dvc-ds4se/se-benchmarking/traceability/data...,38.018795,0.025629,0.820864,0.179136,674.813660,0.001480,0.180110,0.0
